In [ ]:
import torch
import numpy as np

dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 16, 4, 3, 1

# Create random input and output data

x_numpy = np.array(    [[0., 0., 0., 1.],
                        [1., 0., 0., 1.],
                        [0., 1., 0., 1.],
                        [0., 0., 1., 1.],
                        [1., 1., 0., 1.],
                        [1., 0., 1., 1.],
                        [0., 1., 1., 1.],
                        [1., 1., 1., 1.],
                        [0., 0., 0., 0.],
                        [1., 0., 0., 0.],
                        [0., 1., 0., 0.],
                        [0., 0., 1., 0.],
                        [1., 1., 0., 0.],
                        [1., 0., 1., 0.],
                        [0., 1., 1., 0.],
                        [1., 1., 1., 0.]])

x = torch.from_numpy(x_numpy).float()
print(x)


In [ ]:
y_numpy = np.array(     [[1.],
                         [1.],
                         [1.],
                         [1.],
                         [1.],
                         [1.],
                         [1.],
                         [1.],
                         [0.],
                         [0.],
                         [0.],
                         [0.],
                         [0.],
                         [0.],
                         [0.],
                         [0.]])

y = torch.from_numpy(y_numpy).float()

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

In [ ]:
learning_rates = [1e-4, 1e-3, 1e-2]
H = [5, 10, 20]
activation_functions = [torch.nn.Tanh(), torch.nn.ReLU()]

In [ ]:
parameters_list = []
loss_lists = []
for learning_rate in learning_rates:
  for h in H:
    for activ in activation_functions:

      wandb.init(
      project="intro-to-wandb", 
      name=f"experiment_lr_{learning_rate}_h_{h}_activ_{activ._get_name()}", 
      config={
        "learning_rate": learning_rate,
        "architecture": "MLP",
        "dataset": "Logical gates",
        "activation function": activ._get_name(),
        "number_of_neurons" : h,
        "epochs": 100})

      config = wandb.config

      model = torch.nn.Sequential(
      torch.nn.Linear(D_in, h),
      activ,
      torch.nn.Linear(h, D_out)
      )
      
      loss_fn = torch.nn.MSELoss(reduction='sum')
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
      loss_list = []
      for t in range(config.epochs):
        y_pred = model(x)
        loss = loss_fn(y_pred, y)

        loss_list.append(loss.item())
        train_metrics = {"train/epoch": t,
                        "train/loss": loss.item()}
       


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({**train_metrics})

      loss_lists.append(loss_list)
wandb.finish()


In [ ]:
import matplotlib.pyplot as plt

number_of_loss_list = 0
plt.plot(loss_lists[number_of_loss_list], label = 'loss')
plt.legend()
plt.show()
print("Loss in last iteration wherer lr = {}: {}".format(learning_rates[number_of_loss_list], loss_lists[number_of_loss_list][-1]))

### Zadania

Analogicznie do poprzednich, sprobować innych parametrow i zobaczyc na stronie jak to działa.